In [ ]:
from datetime import datetime

from arthurai import ArthurAI
from arthurai.common.constants import InputType, OutputType, Stage
import joblib
import numpy as np
import pandas as pd
import pytz

In [ ]:
import sys
sys.path.append("..")
from model_utils import load_datasets

In this guide, we'll use the credit dataset (and a pre-trained model) to onboard a new model to the Arthur platform. We'll walk through registering the model using a sample of the training data. This is an example of a batch model.

#### Set up connection
Supply your API Key below to autheticate with the platform.

In [ ]:
# credentials are being passed to the client via environment variables
connection = ArthurAI()

## Create Model

We'll instantiate a model object with a small amount of metadata about the model input and output types. Then, we'll use a sample of the training data to register the full data schema for this Tabular model.

In [ ]:
arthur_model = connection.model(partner_model_id=f"CreditRisk_Batch_QS-{datetime.now().strftime('%Y%m%d%H%M%S')}",
                                display_name="Credit Risk Batch",
                                input_type=InputType.Tabular,
                                output_type=OutputType.Multiclass,
                                is_batch=True)

In [ ]:
(X_train, Y_train), (X_test, Y_test) = load_datasets("../fixtures/datasets/credit_card_default.csv")

In [ ]:
Y_train.head()

In [ ]:
X_train.head()

In [ ]:
# load our pre-trained classifier so we can generate predictions
sk_model = joblib.load("../fixtures/serialized_models/credit_model.pkl")

# get model predictions
preds = sk_model.predict_proba(X_train)
X_train["prediction_1"] = preds[:, 1]

# # get ground truth labels
X_train["gt"] = Y_train

We need to register the schema for the outputs of the model: what will a typical prediction look like and what will a typical ground truth look like? What names, shapes, and datatypes should Arthur expect for these objects?

We'll do this all in one step with the *.build()* method. All we need to supply is:
  * the training dataframe
  * the mapping that related predictions to ground truth
  * positive predicted attribute label
  
Our classifier will be making predictions about class *0* and class *1* and will return a probability score for each class. Therefore, we'll set up a name *prediction_0* and a name *prediction_1*. Additionally, our groundtruth will be either a 0 or 1, but we'll always represent ground truth in the one-hot-endoded form. Therefore, we create two fields called *gt_0* and *gt_1*. We link these all up in a dictionary and pass that to the model.  

In [ ]:
# Map our prediction attribute to the ground truth value
prediction_to_ground_truth_map = {
    "prediction_1": 1
}

arthur_model.build(X_train, 
                   ground_truth_column="gt",
                   pred_to_ground_truth_map=prediction_to_ground_truth_map)

Before saving, you can also review your model to make sure everything is correct from the output of `arthur_model.build()` or via `arthur_model.review()`.

When saving your model, the data is saved as the reference set, which is used as the baseline data for tracking data drift. Often, this is the training data for the associated model. Our reference dataset should include:
  * inputs 
  * ground truth
  * model predictions
  
This way, Arthur can monitor for drift and stability in all of these aspects. 

If you've already created your model, you can fetch it from the Arthur API. Retrieve a Model ID from the output of the `arthur_model.save()` call below, or the URL of your model page in the Arthur Dashboard.

In [ ]:
model_id = arthur_model.save()
with open("quickstart_model_id.txt", "w") as f:
    f.write(model_id)

In [ ]:
# you can fetch a model by ID. for example pull the last-created model:
# with open("quickstart_model_id.txt", "r") as f:
#     model_id = f.read()
# arthur_model = connection.get_model(model_id)

## Sending Batches of Inferences

Load test data and trained model. Let's familiarize ourselves with the data and the model.


The predictions/scores from your model should match the column names in the registered schema. If we take a look above at *arthur_model.review()* we'll recall that columns we created correspond to the clasiffier's output probabilities over the classes ("prediction_1" and "prediction_0") and the corresponding ground truth over the possible clases in one-hot form ("gt_1" and "gt_0").

Aside from these model-specific columns, there are two standard inputs which are needed to indentify inferences.
* First, each inference needs a unique identifier so that it can later be joined with ground truth. Include a column named **partner_inference_id** and ensure these IDs are unique across batches. For example, if you run predictions across your customer base on a daily-batch cadence, then a unique identfier could be composed of your customer_id plus the date.   
* Second, each inference needs an **inference_timestamp** and these don't have to be unique.

We'll use our clasifier to score a batch of inputs and then assemble those inputs and predictions into a dataframe with the matching column names.

In [ ]:
from uuid import uuid4

num_batches = 20
batch_ids = []

for i in range(num_batches):
    batch_size=np.random.randint(1000, 5000)
    batch_id = f"batch_{str(uuid4()).split('-')[1]}"
    batch_ids.append(batch_id)

    # generate a small batch of rows from the test set, create unique id for each row
    batch_df = X_test.sample(batch_size)
    inference_ids = [f"{batch_id}-inf_{i}" for i in batch_df.index]
    
    # calculate predictions on those rows, fetch ground truth for those rows
    batch_predictions = sk_model.predict_proba(batch_df)
    batch_ground_truths = Y_test[batch_df.index]
    
    # need to include model prediction columns, and partner_inference_id
    batch_df["prediction_1"] = batch_predictions[:, 1]
    
    # assemble the inference-wise groundtruth and upload
    ground_truth_df = pd.DataFrame({'gt': batch_ground_truths})

    arthur_model.send_inferences(batch_df, batch_id=batch_id, partner_inference_ids=inference_ids)
    arthur_model.update_inference_ground_truths(ground_truth_df, partner_inference_ids=inference_ids)

Realistically, there will be some delay before you have ground truth for your model's predictions. Whether that ground truth is accessible after one minute or one year, the *update_inference_ground_truths()* method can be called at any later time. The ground truth (labels) will joined with their corresponding predictions to yield accuracy measures. 